<a href="https://colab.research.google.com/github/baluvanan/CNN_projects/blob/development/Transfer_learning_dog_cat_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Activation, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import TensorBoard
import time

MODEL_NAME = 'cats_dogs_RGB_CNN-{}'.format(int(time.time()))
tensorboard = TensorBoard(log_dir="./logs/{}".format(MODEL_NAME))
  
from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)
local_weights_file = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False

In [2]:
pre_trained_model.summary()

67[0][0]              
__________________________________________________________________________________________________
conv2d_69 (Conv2D)              (None, 7, 7, 192)    147456      average_pooling2d_6[0][0]        
__________________________________________________________________________________________________
batch_normalization_60 (BatchNo (None, 7, 7, 192)    576         conv2d_60[0][0]                  
__________________________________________________________________________________________________
batch_normalization_63 (BatchNo (None, 7, 7, 192)    576         conv2d_63[0][0]                  
__________________________________________________________________________________________________
batch_normalization_68 (BatchNo (None, 7, 7, 192)    576         conv2d_68[0][0]                  
__________________________________________________________________________________________________
batch_normalization_69 (BatchNo (None, 7, 7, 192)    576         conv2d_69[0][0]      

In [3]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [4]:
from tensorflow.keras.optimizers import RMSprop
x = last_output
#x = layers.Conv2D(128,(3,3), activation="relu",padding='same')(x)
#x = layers.Activation(activation=)(x)
#x = layers.MaxPool2D((2,2))(x)

#x = layers.Conv2D(128,(3,3), activation="relu",padding='same')(x)
#x = layers.MaxPool2D((2,2))(x)
# Flatten the output layer to 1 dimension
x = layers.Flatten()(x)
# Add a fully connected layer with 1,024 hidden units and ReLU activation

#x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
#x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x)

model.compile(optimizer = 'adam', 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])


In [5]:
model.summary()

ation_44[0][0]     
__________________________________________________________________________________________________
conv2d_45 (Conv2D)              (None, 7, 7, 160)    179200      activation_44[0][0]              
__________________________________________________________________________________________________
batch_normalization_45 (BatchNo (None, 7, 7, 160)    480         conv2d_45[0][0]                  
__________________________________________________________________________________________________
activation_45 (Activation)      (None, 7, 7, 160)    0           batch_normalization_45[0][0]     
__________________________________________________________________________________________________
conv2d_41 (Conv2D)              (None, 7, 7, 160)    122880      mixed4[0][0]                     
__________________________________________________________________________________________________
conv2d_46 (Conv2D)              (None, 7, 7, 160)    179200      activation_45[0][0]     


# Fetch Data

In [6]:
base_dir = './cats_and_dogs_filtered/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')

In [7]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))


Found 2681 images belonging to 2 classes.
Found 319 images belonging to 2 classes.


In [8]:
history = model.fit(train_generator,validation_data=validation_generator, batch_size=32, epochs=10,callbacks=[tensorboard])

Epoch 1/10
135/135 [==============================] - 41s 246ms/step - loss: 0.3416 - accuracy: 0.8316 - val_loss: 0.1227 - val_accuracy: 0.9530
Epoch 2/10
135/135 [==============================] - 18s 133ms/step - loss: 0.1784 - accuracy: 0.9248 - val_loss: 0.1992 - val_accuracy: 0.9310
Epoch 3/10
135/135 [==============================] - 18s 135ms/step - loss: 0.1913 - accuracy: 0.9196 - val_loss: 0.1587 - val_accuracy: 0.9561
Epoch 4/10
135/135 [==============================] - 19s 137ms/step - loss: 0.1421 - accuracy: 0.9453 - val_loss: 0.1254 - val_accuracy: 0.9530
Epoch 5/10
135/135 [==============================] - 19s 141ms/step - loss: 0.1489 - accuracy: 0.9499 - val_loss: 0.1403 - val_accuracy: 0.9530
Epoch 6/10
135/135 [==============================] - 19s 138ms/step - loss: 0.1664 - accuracy: 0.9496 - val_loss: 0.2101 - val_accuracy: 0.9467
Epoch 7/10
135/135 [==============================] - 20s 145ms/step - loss: 0.1767 - accuracy: 0.9389 - val_loss: 0.1287 - val_ac

In [9]:
model.save("{}.h5".format(MODEL_NAME))

In [10]:
MODEL_NAME

'cats_dogs_RGB_CNN-1613859215'